In [ ]:
from PIL import Image

from vllm import LLM

from llm.settings import DOCUMENTS_DIR, MODEL_NAME

images = [
    Image.open(image_path).convert("RGB") for image_path in DOCUMENTS_DIR.iterdir()
]

llm = LLM(
    model=MODEL_NAME,
    gpu_memory_utilization=0.9,
    max_num_seqs=2,
    max_model_len=4096,
    mm_processor_kwargs={"min_pixels": 28 * 28, "max_pixels": 1280 * 28 * 28},
    disable_mm_preprocessor_cache=True,
)

/teamspace/studios/this_studio/demo-ai-data-eng/llm/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 04-11 08:17:19 [__init__.py:239] Automatically detected platform cuda.


2025-04-11 08:17:21,455	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 04-11 08:17:35 [config.py:600] This model supports multiple tasks: {'embed', 'generate', 'reward', 'classify', 'score'}. Defaulting to 'generate'.
INFO 04-11 08:17:35 [config.py:1780] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 04-11 08:17:38 [core.py:61] Initializing a V1 LLM engine (v0.8.3) with config: model='Qwen/Qwen2.5-VL-3B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-VL-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_tra

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


INFO 04-11 08:17:41 [gpu_model_runner.py:1258] Starting to load model Qwen/Qwen2.5-VL-3B-Instruct...
WARNING 04-11 08:17:41 [vision.py:101] Current `vllm-flash-attn` has a bug inside vision module, so we use xformers backend instead. You can run `pip install flash-attn` to use flash-attention backend.
INFO 04-11 08:17:41 [config.py:3334] cudagraph sizes specified by model runner [1, 2, 4, 8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120, 128, 136, 144, 152, 160, 168, 176, 184, 192, 200, 208, 216, 224, 232, 240, 248, 256, 264, 272, 280, 288, 296, 304, 312, 320, 328, 336, 344, 352, 360, 368, 376, 384, 392, 400, 408, 416, 424, 432, 440, 448, 456, 464, 472, 480, 488, 496, 504, 512] is overridden by config [512, 384, 256, 128, 4, 2, 1, 392, 264, 136, 8, 400, 272, 144, 16, 408, 280, 152, 24, 416, 288, 160, 32, 424, 296, 168, 40, 432, 304, 176, 48, 440, 312, 184, 56, 448, 320, 192, 64, 456, 328, 200, 72, 464, 336, 208, 80, 472, 344, 216, 88, 120, 480, 352, 248, 224, 96, 488, 504, 

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.33s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.32s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.32s/it]



INFO 04-11 08:17:44 [loader.py:447] Loading weights took 2.75 seconds
INFO 04-11 08:17:45 [gpu_model_runner.py:1273] Model loading took 7.1557 GiB and 3.431920 seconds
INFO 04-11 08:17:45 [gpu_model_runner.py:1542] Encoder cache will be initialized with a budget of 8192 tokens, and profiled with 2 video items of the maximum feature size.
INFO 04-11 08:18:17 [backends.py:416] Using cache directory: /home/zeus/.cache/vllm/torch_compile_cache/703ef19df1/rank_0_0 for vLLM's torch.compile
INFO 04-11 08:18:17 [backends.py:426] Dynamo bytecode transform time: 21.47 s
INFO 04-11 08:18:19 [backends.py:115] Directly load the compiled graph for shape None from the cache
INFO 04-11 08:18:32 [monitor.py:33] torch.compile takes 21.47 s in total
INFO 04-11 08:18:34 [kv_cache_utils.py:578] GPU KV cache size: 269,040 tokens
INFO 04-11 08:18:34 [kv_cache_utils.py:581] Maximum concurrency for 4,096 tokens per request: 65.68x
INFO 04-11 08:19:06 [gpu_model_runner.py:1608] Graph capturing finished in 32 se

In [ ]:
from vllm import SamplingParams
from vllm.sampling_params import GuidedDecodingParams

from llm.parser.prompts import INSTRUCTION
from llm.parser.schemas import Invoice

json_schema = Invoice.model_json_schema()

guided_decoding_params = GuidedDecodingParams(json=json_schema)
sampling_params = SamplingParams(
    # guided_decoding=guided_decoding_params,
    max_tokens=2048,
    temperature=0,
    # stop=["<|im_end|>"]
)

formatted_prompt = (
    "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n"
    f"<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|>"
    f"{INSTRUCTION}<|im_end|>\n"
    "<|im_start|>assistant\n"
)

print(formatted_prompt)

inputs = [
    {
        "prompt": formatted_prompt,
        "multi_modal_data": {"image": image},
    }
    for image in images
]

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>Extract the data from this invoice.
Return your response as a valid JSON object.
Here's an example of the expected JSON output: 

{
    "invoiced_date": 09/04/2025  #format DD/MM/YYYY
    "due_date": 09/04/2025  #format DD/MM/YYYY
    "from_info": {
        "email": "jeremya@gmail.com",
        "phone_number": "+33645789564",
        "address": {
            "street": "Chemin des boulangers",
            "city": "Bourges",
            "country": FR # 2 letters country
        },
    "to_info": {
        "email": "igordosgor@gmail.com",
        "phone_number": "+33645789564",
        "address": {
            "street": "Chemin des boulangers",
            "city": "New York",
            "country": US
        },
    }
    "amount": {
        "sub_total": 1450.4 # Before taxes
        "total": 1,740.48 # After taxes
        "vat": 0.2 # Pourcentage
        "currency": USD #

In [3]:
outputs = llm.generate(inputs, sampling_params=sampling_params)
outputs[0].outputs[0].text

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Processed prompts: 100%|██████████| 10/10 [00:42<00:00,  4.21s/it, est. speed input: 223.73 toks/s, output: 63.62 toks/s]


'```json\n{\n    "invoiced_date": "24-May-2020",\n    "due_date": "",\n    "from_info": {\n        "email": "barton@example.net",\n        "phone_number": "+9890846146",\n        "address": {\n            "street": "1954 Wilson Mall",\n            "city": "Turnerside",\n            "country": "MH 21563 US"\n        }\n    },\n    "to_info": {\n        "email": "carmenreeves@example.com",\n        "phone_number": "",\n        "address": {\n            "street": "",\n            "city": "",\n            "country": ""\n        }\n    },\n    "amount": {\n        "sub_total": 1132.97,\n        "total": 1139.38,\n        "vat": 0.359,\n        "currency": "USD"\n    }\n}\n```'

In [4]:
for output in outputs:
    print((output.outputs[0].text))

```json
{
    "invoiced_date": "24-May-2020",
    "due_date": "",
    "from_info": {
        "email": "barton@example.net",
        "phone_number": "+9890846146",
        "address": {
            "street": "1954 Wilson Mall",
            "city": "Turnerside",
            "country": "MH 21563 US"
        }
    },
    "to_info": {
        "email": "carmenreeves@example.com",
        "phone_number": "",
        "address": {
            "street": "",
            "city": "",
            "country": ""
        }
    },
    "amount": {
        "sub_total": 1132.97,
        "total": 1139.38,
        "vat": 0.359,
        "currency": "USD"
    }
}
```
```json
{
    "invoiced_date": "25-May-2004",
    "due_date": "22-Jun-2015",
    "from_info": {
        "email": "melvin40@example.net",
        "phone_number": "+0287309623",
        "address": {
            "street": "16424 Timothy Mission",
            "city": "Markville",
            "country": "AK 58294 US"
        }
    },
    "to_info": {
  

In [5]:
import json

output = outputs[0].outputs[0].text
start = output.find("{")
end = output.rfind("}") + 1  # +1 to include the closing brace

# Extract the JSON part
json_str = output[start:end]
json_obj = json.loads(json_str)
json_obj

{'invoiced_date': '24-May-2020',
 'due_date': '',
 'from_info': {'email': 'barton@example.net',
  'phone_number': '+9890846146',
  'address': {'street': '1954 Wilson Mall',
   'city': 'Turnerside',
   'country': 'MH 21563 US'}},
 'to_info': {'email': 'carmenreeves@example.com',
  'phone_number': '',
  'address': {'street': '', 'city': '', 'country': ''}},
 'amount': {'sub_total': 1132.97,
  'total': 1139.38,
  'vat': 0.359,
  'currency': 'USD'}}